In [ ]:
import pdfplumber
import pandas as pd
import re

In [ ]:
def clean_text(text):
    """Remove caracteres especiais e normaliza espaços"""
    text = re.sub(r'[\*\#]', '', text)  # Remove negrito e marcadores
    text = re.sub(r'\s+', ' ', text).strip()
    return text

# Regex melhorado para cargos (inclui TÉCNICO e variações)
position_pattern = re.compile(
    r'(TÉCNICO)\s*[-\s]\s*([^\-\n]+?)\s*$',
    re.IGNORECASE
)

# Regex para local
location_pattern = re.compile(
    r'(TRIBUNAL DE JUSTIÇA|COMARCAS DO ESTADO).*?(\([A-Z]{2}\))?$'
)

# Regex para candidatos (mais flexível)
candidate_pattern_ob = re.compile(
    r'^(\d+[/-]?\d*[/-]?\d+)\s+([^\d]+?)\s+(\d+[,.]\d+|\d+)\s+(\d+[,.]\d+|\d+)\s+'
    r'(\d+[,.]\d+|\d+)\s+(\d+[,.]\d+|\d+)\s+(\d+[,.]\d+|\d+)\s+(.+)$'
)
candidate_pattern_dis = re.compile(
     r'^(\d+)\s+([^\d]+?)\s+(\d+[,.]\d+)\s+(.+)$'
)

data_obje = []
data_disc = []
current_position = None
current_location = None

In [ ]:
with pdfplumber.open("prova_objetiva.pdf") as pdf:
    for page in pdf.pages:
        text = page.extract_text()
        
        for line in text.split('\n'):
            line = clean_text(line)
            
            # Pular linhas irrelevantes
            if any(term.lower() in line.lower() for term in [
                "instituto", "página", "resultado", "edital", "concurso"
            ]) and not any(term in line for term in ["TRIBUNAL", "COMARCAS"]):
                continue
                
            # Verificar cargo (ANALISTA ou TÉCNICO)
            position_match = position_pattern.search(line)
            if position_match:
                tipo = position_match.group(1).upper()
                especialidade = position_match.group(2).strip()
                current_position = f"{tipo} - {especialidade}"
                # print(f"Cargo detectado: {current_position}")  # Debug
                continue
                
            # Verificar localização
            location_match = location_pattern.search(line)
            if location_match:
                current_location = line.strip()
                # print(f"Local detectado: {current_location}")  # Debug
                continue
                
            # Verificar candidato
            candidate_match = candidate_pattern_ob.match(line)
            if candidate_match and current_position and current_location:
                inscricao = candidate_match.group(1)
                nome = candidate_match.group(2).strip()
                
                # Adicionar à lista de dados
                data_obje.append([
                    inscricao,
                    nome,
                    float(candidate_match.group(3).replace(',', '.')),
                    float(candidate_match.group(4).replace(',', '.')),
                    float(candidate_match.group(5).replace(',', '.')),
                    float(candidate_match.group(6).replace(',', '.')),
                    float(candidate_match.group(7).replace(',', '.')),
                    candidate_match.group(8),
                    current_position,
                    current_location
                ])

In [ ]:

# Criar DataFrame
columns_ob = [
    "Inscrição", "Nome", "Língua Portuguesa", 
    "Conhecimentos Transversais", "História e Geografia",
    "Conhecimentos Específicos", "Nota Objetiva", 
    "Situação", "Cargo", "Local"
]

df = pd.DataFrame(data_obje, columns=columns_ob)


In [ ]:
with pdfplumber.open("prova_discursiva.pdf") as pdf:
    for page in pdf.pages:
        text = page.extract_text()
        
        for line in text.split('\n'):
            line = clean_text(line)
            # Pular linhas irrelevantes
            if any(term.lower() in line.lower() for term in [
                "instituto", "página", "resultado", "edital", "concurso", "candidatos", "Consulplan"
            ]) and not any(term in line for term in ["TRIBUNAL", "COMARCAS"]):
                continue

                
            # Verificar cargo (ANALISTA ou TÉCNICO)
            position_match = position_pattern.search(line)
            if position_match:
                tipo = position_match.group(1).upper()
                especialidade = position_match.group(2).strip()
                current_position = f"{tipo} - {especialidade}"
                # print(f"Cargo detectado: {current_position}")  # Debug
                continue
                
            # Verificar localização
            location_match = location_pattern.search(line)
            if location_match:
                current_location = line.strip()
                # print(f"Local detectado: {current_location}")  # Debug
                continue
                
            
            # Verificar candidato
            candidate_match = candidate_pattern_dis.match(line)
            if candidate_match and current_position and current_location:
                inscricao = candidate_match.group(1)
                nome = candidate_match.group(2).strip()
                
                # Adicionar à lista de dados
                data_disc.append([
                    inscricao,
                    nome,
                    float(candidate_match.group(3).replace(',', '.')),
                    candidate_match.group(4),
                    current_position,
                    current_location
                ])

In [ ]:
# Criar DataFrame
columns_dis = [
    "Inscrição", "Nome", "Prova Discursiva", 
    "Situação", "Cargo", "Local"
]

df2 = pd.DataFrame(data_disc, columns=columns_dis)

In [ ]:
# Pós-processamento
df['Cargo'] = df['Cargo'].str.replace('JUSTICA', 'JUSTIÇA')  # Corrige ortografia
df['Cargo'] = df['Cargo'].str.replace('PEDIAIRA', 'PEDIATRA')  # Corrige ortografia
df2['Cargo'] = df2['Cargo'].str.replace('JUSTICA', 'JUSTIÇA')  # Corrige ortografia
df2['Cargo'] = df2['Cargo'].str.replace('PEDIAIRA', 'PEDIATRA')  # Corrige ortografia

In [ ]:
# Salvar resultados
df.to_csv("prova_objetiva_compilado.csv", index=False, encoding='utf-8-sig')
df2.to_csv("prova_discursiva_compilado.csv", index=False, encoding='utf-8-sig')

print(f"\nTotal de registros extraídos: {len(df)} {len(df2)}")

In [ ]:
# Merge (juntar) os DataFrames
df_final = pd.merge(df, df2, on="Inscrição", how="inner")

# Criar coluna com a soma das notas
df_final["Nota Final"] = df_final["Nota Objetiva"] + df_final["Prova Discursiva"]

df_final = df_final.drop(columns=["Situação_x", "Cargo_x", "Local_x", "Nome_y"])

df_final = df_final.sort_values(by="Nota Final", ascending=False)

# Salvar em CSV (opcional)
df_final.to_csv("dados_finais_tjro.csv", index=False, encoding='utf-8-sig')


print(f"\nTotal de registros extraídos: {len(df)} {len(df2)}  {len(df_final)}")

df_final.head(10)